In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you 
# create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
PATH = '/kaggle/input/twitter-sentiment-analysis-hatred-speech/'
df = pd.read_csv(PATH + "train.csv", index_col=0)
# df_test = pd.read_csv(PATH + "test.csv")

In [4]:
df_train, df_val = train_test_split(df, random_state=42, test_size=0.3)
df_train.shape, df_val.shape

In [5]:
df_train.label.value_counts()

In [6]:
df_val.label.value_counts()

In [7]:
max_words = 4000
max_len = 10
num_classes = 1

# Training
epochs = 10
batch_size = 512
print_batch_n = 100

# Предобработка

In [8]:
!pip install -q stop-words pymorphy2

In [9]:
import re
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('omw-1.4')

In [10]:
sw = set(get_stop_words("en"))
sw.add('user')  
puncts = set(punctuation)
morpher = MorphAnalyzer()
lemmatizer = WordNetLemmatizer()

In [11]:
# sw
morpher.parse('бегали')[0].normal_form

In [12]:
lemmatizer.lemmatize("rocks")

In [13]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
    txt = re.sub("not\s", "not", txt)
#     txt = re.sub("(.\\x[0-9]*.|\Z)(\\x[0-9]*.|\Z|)", " ", txt)
#     txt = "".join(c for c in txt if c not in puncts)
#     txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    txt = [lemmatizer.lemmatize(word) for word in txt.split() if word not in sw]
    return " ".join(txt)

In [14]:
a = df_train.tweet.iloc[0]
a

In [15]:
df_train['tweet'].iloc[:1].apply(preprocess_text).values

In [16]:
from tqdm import tqdm 
tqdm.pandas()

df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
df_val['tweet'] = df_val['tweet'].progress_apply(preprocess_text)

In [17]:
train_corpus = " ".join(df_train["tweet"])
train_corpus = train_corpus.lower()

In [18]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)
tokens[:5]

Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [19]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [20]:
from nltk.probability import FreqDist

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]  # вычитание 1 для padding
len(tokens_filtered_top)

In [21]:
tokens_filtered_top[:10]

In [22]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
# vocabulary

In [23]:
import numpy as np


def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [24]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["tweet"]])
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["tweet"]])

In [25]:
x_train.shape, x_val.shape

In [26]:
df_train.tweet.iloc[0]

In [27]:
x_train[0]

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [30]:
class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size=2000, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, ht = self.lstm(x)
       
        if self.use_last:
            last_tensor = lstm_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(lstm_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        # print(out.shape)
        return torch.sigmoid(out)

In [31]:
from torch.utils.data import DataLoader, Dataset


class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [32]:
train_dataset = DataWrapper(x_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_val, df_val['label'].values)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [33]:
for x, l in train_loader:
    print(x.shape)
    print(l.shape)
    print(l[0])
    break

In [34]:
# model = Net(vocab_size=max_words)
model = LSTMFixedLen(vocab_size=max_words)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [35]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [37]:
from torchmetrics import F1Score
# f1 = F1Score(num_classes=2)
f1 = F1Score().to(device)

In [38]:
len(train_loader)

In [39]:
model = model.to(device)
model.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    running_items, running_right = 0.0, 0.0
    f1_res = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        f1_res += f1(outputs, labels)
        
    # выводим статистику о процессе обучения
    model.eval()
    
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
            f'Step [{i + 1}/{len(train_loader)}]. ' \
            f'Loss: {loss:.3f}. ' \
            f'F1: {f1_res / len(train_loader):.4f}. ' \
            f'Acc: {running_right / running_items:.3f}', end='. ')
    running_items, running_right = 0.0, 0.0
    f1_res = 0.0
    train_loss_history.append(loss)

        # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    test_f1_res = 0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = model(data[0].to(device))
        
        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчет метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
        test_f1_res += f1(test_outputs, test_labels)
    
    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}. ' \
          f'Test F1: {test_f1_res / len(val_loader):.4f}. ' \
          f'Test acc: {test_running_right / test_running_total:.3f}')
    
    model.train()
        
print('Training is finished!')

In [40]:
import matplotlib.pyplot as plt

plt.title('Loss history')
plt.grid(True)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.plot(train_loss_history, label='train')
plt.plot(test_loss_history, label='test')
plt.legend();